# Preamble
You can put any code here, it won't be included in the deployment. Also take care, that the deployments cannot interpret magical lines starting with `!`.

In [ ]:
#!skip
!pip3 install opencv-python matplotlib ultralytics
!pip3 install git+https://github.com/hollosigergely/aitoolbox_support_library.git

# Setup

Load OpenCV for image processing and ultralytics for YOLO.

In [ ]:
from ultralytics import YOLO
import cv2
from aitoolbox_support_lib.context import Context
from aitoolbox_support_lib.datatypes.image import Image

In [ ]:
#!skip
import matplotlib.pyplot as plt

In [ ]:
#!skip

# Define test input with example
# Make sure its colorspace is converted, CV uses BGR, we need RGB
cv_img = cv2.cvtColor(cv2.imread('assets/example.jpg'),cv2.COLOR_BGR2RGB)
Context.get().get_sources().set("image",Image(cv_img))

Open YOLO model. Use the checkpoint files from the [pretrained models](https://docs.ultralytics.com/models/yolov8/#supported-tasks-and-modes), or the model trained on custom dataset by [train.ipynb](train.ipynb).

In [ ]:
model = YOLO('yolov8n.pt')

# Service

Service inputs:

* image (*Image*): the image to detect on 
* test_mode (*bool*): if True, the service returns an image with the detected boxes

In [ ]:
image = Context.get().get_sources().get("image",Image)
test_mode = Context.get().get_sources().get("test_mode",bool,default_val = False)
image = image.to_numpy()

In [ ]:
result = model(image)

In [ ]:
#!skip

plt.imshow(result[0].plot())

In `test mode` we return the single image with the detections visualized. Otherwise, we respond with the detections as bounding boxes.

In [ ]:
result[0].boxes.xyxy.cpu().numpy()

In [ ]:
destination = Context.get().get_destinations()
if test_mode: 
    destination.enable_single_mimetype(True)
    destination.set("resp",Image(result[0].plot()))
else:
    destination.set("boxes", result[0].boxes.xyxy.cpu().numpy())
    destination.set("cls", result[0].boxes.cls.cpu().numpy())

Test response format:

In [ ]:
#!skip

import json
json.loads(destination.serialize())